In [1]:
#DataFrame Manipulation
import numpy as np
import pandas as pd

#Web Scrapping
import requests
from bs4 import BeautifulSoup
import xml
from pandas.io.json import json_normalize

In [2]:

from geopy.geocoders import Nominatim
import folium

In [5]:
geolocator = Nominatim()
loc = geolocator.geocode('New Delhi, India')
lat = loc.latitude
long = loc.longitude

In [6]:
lat

28.6141793

In [7]:
long

77.2022662

In [8]:
df_geo = pd.DataFrame({'Spot': ['Red Fort','Qutub Minar',"Humayun's Tomb","India Gate"], 'Latitude': [28.6562,28.5245,28.5933,28.6129],
                       'Longitude': [77.2410,77.1855,77.2507,77.2295]}, columns = ['Spot','Latitude','Longitude'])

In [9]:
df_geo

,Spot,Latitude,Longitude
0,Red Fort,28.6562,77.2410
1,Qutub Minar,28.5245,77.1855
2,Humayun's Tomb,28.5933,77.2507
3,India Gate,28.6129,77.2295


In [10]:
map_New_Delhi = folium.Map(location=[lat, long], zoom_start=10)

# add markers to map
for lat, lng, spt in zip(df_geo['Latitude'], df_geo['Longitude'], df_geo['Spot']):
    label = '{}'.format(spt)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_New_Delhi)  
    
map_New_Delhi

In [11]:
client_id = "HAVUU0HMOGTR3CBAKB3QWT1ZJDDBBOWBC0U0K1JGZTDTXMXA"
client_secret = "XWORBASJJOG0KQAH04NNPZBEGZDSKY2JX5225MPEONUNS2VY"

CLIENT_ID = client_id # your Foursquare ID
CLIENT_SECRET = client_secret # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HAVUU0HMOGTR3CBAKB3QWT1ZJDDBBOWBC0U0K1JGZTDTXMXA
CLIENT_SECRET:XWORBASJJOG0KQAH04NNPZBEGZDSKY2JX5225MPEONUNS2VY


In [12]:
spot_latitude = df_geo['Latitude'][0] # spot latitude value
spot_longitude = df_geo['Longitude'][0] # spot longitude value

spot_name = df_geo['Spot'][0] # spot name

print('Latitude and longitude values of {} are {}, {}.'.format(spot_name, 
                                                               spot_latitude, 
                                                               spot_longitude))

Latitude and longitude values of Red Fort are 28.6562, 77.241.


In [13]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    spot_latitude, 
    spot_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=HAVUU0HMOGTR3CBAKB3QWT1ZJDDBBOWBC0U0K1JGZTDTXMXA&client_secret=XWORBASJJOG0KQAH04NNPZBEGZDSKY2JX5225MPEONUNS2VY&v=20180605&ll=28.6562,77.241&radius=500&limit=100'

In [14]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d04f93623bb8e00388d355a'},
 'response': {'headerLocation': 'Jama Masjid',
  'headerFullLocation': 'Jama Masjid, Delhi',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 28.660700004500004,
    'lng': 77.24611855937579},
   'sw': {'lat': 28.651699995499992, 'lng': 77.23588144062421}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bf214b83506ef3b1d7abd22',
       'name': 'Red Fort | Lal Qila | लाल क़िला | لال قلعہ (Red Fort (Lal Qila))',
       'location': {'address': 'Mahatma Gandhi Marg',
        'lat': 28.655758930875194,
        'lng': 77.2419548034668,
        'labeledLatLngs': [{'label': 'display',
          'lat': 28.655758930875194,
          'lng': 77.2419548034668}],
 

In [15]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Red Fort | Lal Qila | लाल क़िला | لال قلعہ (Re...,Monument / Landmark,28.655759,77.241955
1,Red Fort Archaeological Museum | लाल किला पुरा...,Museum,28.655964,77.237655
2,Lal Qila Metro Station,Metro Station,28.656691,77.236999
3,Azad Hind Bus Stop,Bus Station,28.656135,77.235960


In [17]:

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
scar_venues = getNearbyVenues(names=df_geo['Spot'],
                                   latitudes=df_geo['Latitude'],
                                   longitudes=df_geo['Longitude']
                                  )

Red Fort
Qutub Minar
Humayun's Tomb
India Gate
